# Comparing Counterfactual Explanations methods

This Jupyter notebook is for comparing the Contrastive Explanations Method [https://papers.nips.cc/paper/2018/file/c5ff2543b53f4cc0ad3819a36752467b-Paper.pdf] and the Feasible and Actionable Counterfactual Method (FACE - https://arxiv.org/abs/1909.09369) using K-Nearest Neighbour, Epsilon graphs (using CARLA repo) and Kernel Density Estimator (GraphCounterfactuals).

They will be compared visually on the tabular California Housing Dataset and the MNIST dataset.

This notebook requires python 3.7.8/9

## MNIST Analysis

"To this end, we trained a Neural Network
with two hidden layers of length 10 and ReLU activation functions.
FACE was initialised with w(z) = −loд(z) as the weight function
and the l2-norm as the distance function."

In [ ]:
import sys
sys.path.insert(0, './repoCarla') # location of src 

from carla.data.api import Data
from carla.models.catalog import MLModelCatalog
from carla.models.api import MLModel
from carla.recourse_methods.catalog.face import Face
from carla import Data, MLModel
from carla.recourse_methods import Face, CEM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rand
#import pymc3 as pm
from typing import Union
import math
import time
from sklearn.datasets import fetch_openml, fetch_california_housing
import matplotlib.pyplot as plt
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Normalizer, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import cdist, pdist
from scipy.stats import gaussian_kde
import networkx as nx
from sklearn.neighbors import DistanceMetric

X_MNIST,y_MNIST = fetch_openml('mnist_784', return_X_y=True)
X_train_MNIST, X_test_MNIST, y_train_MNIST, y_test_MNIST = train_test_split(
    X_MNIST, y_MNIST, test_size=0.25, random_state=4)
X_train_MNIST = X_train_MNIST/255.
X_test_MNIST = X_test_MNIST/255.
MNIST_full = fetch_openml('mnist_784')
desc_MNIST = MNIST_full['DESCR']
feature_names_MNIST = MNIST_full['feature_names']
target_names_MNIST = MNIST_full['target_names']
names_MNIST = target_names_MNIST.extend(feature_names_MNIST)
print("NAMES: ")
print(names_MNIST)
dataMNIST = fetch_openml('mnist_784', return_X_y=False, as_frame=True)['frame']
dataMNISTDict = {k: v for k, v in enumerate(dataMNIST)}
# first implement the dataset wrapper
class MNISTData(Data):
    def __init__(self):
        # the dataset could be loaded in the constructor
        self._dataset = fetch_openml('mnist_784', return_X_y=False, as_frame=True)['frame']

    def categoricals(self):
        # this property contains a list of all categorical features (column string names)
        return target_names_MNIST

    def continous(self):
        # this property contains a list of all continuous features (column string names)
        return feature_names_MNIST

    def immutables(self):
        # this property contains a list of features which should not be changed by the recourse method (column string names)
        return []

    def target(self):
        # this property contains the feature name of the target column (string)
        return target_names_MNIST[0]

    def raw(self):
        # this property contains the not encoded and not normalized, raw dataset
        return self._dataset

# second, implement the black-box-model wrapper
class ANNModel(MLModel):
    def __init__(self, data):
        # the constructor can be used to load or build an arbitrary black-box-model
        self._mymodel = load_model("ann2")

        # this property contains a fitted scaler to normalize input data
        # MinMaxScaler from sklearn is predefined, but can be redefined by every other sklearn scaler
        self.scaler = Normalizer().fit(X_train_MNIST)

        # the same is possible for data encoding
        # OneHotEncoder from sklearn with dropped first column for binary data is predefined, but can be
        # changed into any other sklearn encoder.
        self.encoder = OneHotEncoder(handle_unknown='ignore').fit(y_train_MNIST.reshape(-1, 1))

    def feature_input_order(self):
        # this property contains a list of the correct input order of features for the ml model
        return feature_names_MNIST

    def backend(self):
        # this property contains a string with the used backend of the model
        return "pytorch"

    def raw_model(self):
        # this property contains the fitted/ loaded black-box-model
        return self._mymodel

    def predict(self, x: Union[np.ndarray, pd.DataFrame]):
        # the predict function outputs the continous prediction of the model, similar to sklearn.
        return self._mymodel.predict(x)

    def predict_proba(self, x: Union[np.ndarray, pd.DataFrame]):
        # the predict_proba method outputs the prediction as class probabilities, similar to sklearn
        return self._mymodel.predict_proba(x)

def load_model(modeltype):
    if modeltype == "ann2":
        MLPClassifier(alpha=1e-05, hidden_layer_sizes=(10, 10), random_state=1,
              solver='adam')

# after implementing the user-specific model and dataset, the call of the recourse method,
# and the generation of counterfactuals stays the same.
dataset = MNISTData()
model = ANNModel(dataset)
print("-----DATASET-----")
print(fetch_openml('mnist_784', return_X_y=False, as_frame=True)['frame'])
print("-----------------")
# get some factuals from the data to generate counterfactual examples
factuals = dataset.raw().iloc[:10]

# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train_MNIST.shape, y_train_MNIST.shape))
print('Test: X=%s, y=%s' % (X_test_MNIST.shape, y_test_MNIST.shape))
# plot first few images
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	# plot raw pixel data
	plt.imshow(X_train_MNIST[i], cmap=pyplot.get_cmap('gray'))
# show the figure
plt.show()

In [ ]:
dist_threshold = 0.5 # 1.2
density_threshold = 0.008
pred_threshold = 0.999 #  0.9
counterfactuals_FACE_KDE = []
randI = random.randrange(0, len(dataMNIST))
example = dataMNIST.iloc[randI]
example_label = model.predict(example)
data_example, start_node = nearest(example, example_label)
if dist_face(example, data_example) > dist_threshold:
    print('Data point does not meet distance threshold')

def dist_face(x,y):
    return cdist(x.reshape(1,-1), y.reshape(1,-1), metric='euclidean')

def kernel(x,y):
    kernel = gaussian_kde(X.T)
    return kernel((x.reshape(1,-1), y.reshape(1,-1)))
print(dataMNIST['label'])
print(data_same_label)
# nearest neighbour
dist = DistanceMetric.get_metric('euclidean')
def nearest(example, label):
    data_same_label = dataMNIST[dataMNIST['class']==int(label)]
    distances = dist.pairwise(data_same_label[['x','y']], example)
    data_same_label.insert(3, 'distances', distances, True)
    index_min = data_same_label.distances.idxmin()
    nearest_data = np.array([data_same_label.loc[index_min]['x'], data_same_label.loc[index_min]['y']]).reshape(1,-1)
    point = data_same_label.loc[index_min].name
    return nearest_data, index_min

# create weights based on distance threshold
kernel = gaussian_kde(X.T)

m, n = X.shape
w_ij = [0] * int(m*(m-1)*0.5)
edge_weights = []
k = 0
q = 0
for edge_from in range(m):
    k = 1 + edge_from
    for edge_to in range(k,m):
        if dist_face(X[edge_from], X[edge_to])[0][0] < dist_threshold:
            w_ij[q] = -np.log(kernel((X[edge_from] + X[edge_to]) /2)[0] * dist_face(X[edge_from], X[edge_to])[0][0])
        edge_weights.append((edge_from, edge_to, {'weight':w_ij[q]}))
        q += 1

nonzero_edge = []
for i in range(len(edge_weights)):
    if edge_weights[i][2]['weight'] != 0:
        nonzero_edge.append(edge_weights[i])

        
# create graph with nodes that meet density threshold
G = nx.Graph()

G.add_nodes_from(range(len(X)))
G.add_edges_from(nonzero_edge)

# low_density = []
# for i in range(m):
#     if kernel(X[i]) < density_threshold:
#         low_density.append(i)
# G.remove_nodes_from(low_density)

print('nodes that meet density theshold: ', G.number_of_nodes())
print('edges that meet distance theshold: ', G.number_of_edges())

_, ax = plt.subplots(figsize=(12,12))
c = "br"
nx.draw_networkx(G, pos=X, node_color=[c[int(y)] for y in Y], ax=ax)

target_label = 0 if example_label == 1 else 1
target_data = data_df[data_df['label']==int(target_label)]
target_nodes = list(set(list(G.nodes())).intersection(target_data.index))

pred_threshold_fail = []
for i in target_data.index:
    index = target_data.loc[i]
    #np.array([index['x'],index['y']]).reshape(1, -1)
    prob_target = model.predict_proba(np.array(target_data[i]).reshape(1, -1))[0][target_label]
    if prob_target < pred_threshold:
        pred_threshold_fail.append(i)

        
length, path = nx.multi_source_dijkstra(G,target_nodes,target=start_node)
path = path[::-1]

while path[-1] in pred_threshold_fail:
    index = [i for i,node in enumerate(target_nodes) if node == path[-1]][0]
    del target_nodes[index]
          
    length, path = nx.multi_source_dijkstra(G,target_nodes,target=start_node)
    path = path[::-1]
# plot
x_path = []
y_path = []
for node in path:
    x_path.append(data_dict[node][0])
    y_path.append(data_dict[node][1])
counterfactuals_FACE_KDE.append(x_path[-1],y_path[0-1])



#"mode": str ['knn', 'epsilon'],
#                "fraction": float [0 < x < 1]}  determines fraction of data set to be used to
#                                                construct neighbourhood graph
# load recourse model with model specific hyperparameter
rcm_FaceKnn = Face(model, {"mode": "knn", "fraction": 0.1})
rcm_FaceE = Face(model, {"mode": "epsilon", "fraction": 0.1})
# generate counterfactual examples
counterfactuals_FACE_knn = rcm_FaceKnn.get_counterfactuals(factuals)
counterfactuals_FACE_epsilon = rcm_FaceE.get_counterfactuals(factuals)
print("-----COUNTERFACTUALS - FACE (KNN)-----")
print(counterfactuals_FACE_knn)
print("--------------------------------------")

In [ ]:
"""
Initialisation for the Contrastive Explanation Method (CEM).
Paper: https://arxiv.org/abs/1802.07623

Restrictions
------------
- Works currently only on Tensorflow models

Parameters
----------
mlmodel: Black-box-model we want to explore
hyperparams: Parameter for CEM method, with following possibilites
    {
        "batch_size": int default 1,
        "kappa": float default 0.1 Confidence parameter for the attack loss term,
        "init_learning_rate": float default 1e-2 Initial learning rate of optimizer,
        "binary_search_steps": int default 9 Number of steps when attacking,
        "max_iterations": int default 100 Maximum number of iterations for finding a PN or PP,
        "initial_const": int default 10 Initial value to scale that attack loss term,
        "beta": float default 0.9 Regularization constant for the L1 loss term,
        "gamma": float default 0.0 Regularization constant for the optional auto-encoder loss term,
        "mode": str Find pertinant negatives ('PN') or pertinant positives ('PP'),
        "num_classes": int default 2,
        "data_name": str Name of the dataset (important for loading AE weights),
        "ae_params":
            {
                "hidden_layer": List[int] default [20, 10, 7] Sizes of hidden layers,
                "train_ae": bool default True,
                "epochs": int default 5 Number of epochs to train for,
            },
    }
"""
rcm_CEM_PN = CEM(model, {"mode": "PN", "kappa": 0.1, "data_name": "mnist", "ae_params": {"train_ae": False, "hidden_later": [10, 10], "epochs": 5}})
rcm_CEM_PP = CEM(model, {"mode": "PP", "kappa": 0.1, "data_name": "mnist", "ae_params": {"train_ae": False, "hidden_later": [10, 10], "epochs": 5}})
# generate counterfactual examples
counterfactuals_CEM_PN = rcm_CEM_PN.get_counterfactuals(factuals)
counterfactuals_CEM_PP = rcm_CEM_PP.get_counterfactuals(factuals)
print("-----COUNTERFACTUALS - CEM (PP)-----")
print(counterfactuals_CEM_PN)
print("-----COUNTERFACTUALS - CEM (PN)-----")
print(counterfactuals_CEM_PP)
print("--------------------------------------")